# Word Embeddings : le modèle Word2Vec

## Imports

In [2]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [3]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [4]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [5]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [6]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [7]:
len(bigram_phrases.vocab.keys())

7458

Prenons une clé au hasard :

In [22]:
key_= list(bigram_phrases.vocab.keys())[200]
print(key_)

editions_:


Le dictionnaire indique le score de cette coocurrence :

In [23]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [24]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [26]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [27]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [28]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [29]:
print(corpus[:100])

[['a', "*'", 'cw', "'-", 'tm', 'p', '^', '*\'*"', "'", ';', '"*#', "'*", '!'], ["'", '.'], ["'-#)#?"], ["'", '*>.'], ["*/'", "*'", "'", ':', "'", "'*'''##'.", 'h', ';', '5j', ";#'*", 'y', 'i', "('", "'", ',', 'v', "'", '*:', ':', "'", "'-", "'", '#', "'", '*\'#"*\'', '\'"\'\'', '#-', '#', '*', '^^#', '1', '"', ';"', "*'", '>>', 's', '.', 'iiifw', 'v', '.', ',', 'tf', "'^", '-', '1', "--'", '\'--"', '*#', "'*", '#.'], ['c', '-"', 'r', '^>>', 'ywwiis', '".'], ['#', '-', "'", 'rfytjfrtfiyprfc', '!', 'kur7', '*', 'vn', '?,', '16', 'pages', '-', '30', 'c', 'mes', '}?'], ['#', '\\', '<', 'en', 'belgique', 'et', 'allemagne', 'occupee', ')', 'ja', 'l', "l_'", 'etranger', 't', '3s', 'centimes', '*/', 'm', 'juin', '1929', ';', 'edit', '.'], ['****', '*', '*', '*', '*', 'a', '14', 'b', '.'], ['1', '*', '4', 'editions', ':', "'", 'ii', 'mit', '"', '*', 'edit', ',', 'de', 'notl', '.'], [';;#', 'abonnements', '!'], ['agglomeration', 'bruxelloise', '.<<', '..', '..', '(', 'par', 'mon', '.'], ['7', '.

## Entrainement d'un modèle Word2Vec sur ce corpus

In [30]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 0 ns
Wall time: 32.1 ms


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [31]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [37]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [39]:
model.wv["Gouvernementaux"]

KeyError: "Key 'Gouvernementaux' not present"

### Calculer la similarité entre deux termes

In [ ]:
model.wv.similarity("ministre", "roi")

0.6248811

### Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("ministre", topn=10)

[('president', 0.9105064868927002),
 ('secretaire_general', 0.8774850964546204),
 ('president_du_conseil', 0.8763439655303955),
 ('le_ministre', 0.8576363325119019),
 ('ministre_des_finances', 0.8551753759384155),
 ('ministre_des_colonies', 0.8525806665420532),
 ('commissaire_general', 0.8522257208824158),
 ('conseiller', 0.8520253300666809),
 ('premier_ministre', 0.849519670009613),
 ('gouverneur', 0.8465949296951294)]

### Faire des recherches complexes à travers l'espace vectoriel

In [ ]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('berlin', 0.8128865361213684), ('rome', 0.7630703449249268), ('teheran', 0.762186586856842), ('datee_du', 0.7617496848106384), ('tokio', 0.7588318586349487), ('casablanca', 0.75528883934021), ('alger', 0.7517308592796326), ('tanger', 0.7504138350486755), ('moscou', 0.7488557696342468), ('new_york', 0.7335429787635803)]
